# SVM

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from funciones_mecanizados import normalizar
from funciones_mecanizados import cortar

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
# Cargamos el dataset preprocesado.
h1_norm = pd.read_csv('data/panel1_H1_norm.csv')
h2_norm = pd.read_csv('data/panel2_H2_norm.csv')
h3_norm = pd.read_csv('data/panel1_H3_norm.csv')
 
#Elminamos el ruido
nivel_filtro = 0
data = [h1_norm, h2_norm, h3_norm]
for dat in data:
    dat.drop(dat[dat['ENER']<nivel_filtro].index, axis=0, inplace=True)
    
#Nos quedamos con algunos datos del fresado
#corte1 = [0,2]
#corte2 = [2,2.6]

#fresado_h1 = pd.concat([cortar(h1_norm, corte1), cortar(h1_norm, corte2)])
#fresado_h2 = pd.concat([cortar(h2_norm, corte1), cortar(h2_norm, corte2)])
#fresado_h3 = pd.concat([cortar(h3_norm, corte1), cortar(h3_norm, corte2)])

#print('H1:',fresado_h1.shape)
#print('H2:',fresado_h2.shape)
#print('H3:',fresado_h3.shape)
#Juntamos todos los datos en un solo DF
#H = pd.concat([fresado_h1, fresado_h2, fresado_h3])

mecanizados = pd.concat([h1_norm, h2_norm, h3_norm])
mecanizados.reset_index(drop=True, inplace=True)

### TRAIN/TEST SPLIT

In [3]:
columnas = ['RMS','AMP','ENER','P_FRQ','PCNTS','C_FRQ', 'I_FRQ','A_FRQ','R_FRQ',
           'DURATION','ASL','COUN','ABS_ENERGY', 'SIG_STRNGTH','RISE', 'P_MARSE']

In [4]:

train_test_split = StratifiedShuffleSplit(n_splits=1, test_size=.20, random_state=11)

for train_index, test_index in train_test_split.split(mecanizados, mecanizados['Herramienta']):
    train = mecanizados.iloc[train_index]
    test = mecanizados.iloc[test_index]

In [5]:
#Datos para entrenar y probar el modelo.
y_train = train.Herramienta
y_test = test.Herramienta

X_train = train[columnas]
X_test = test[columnas]

### Modelos

In [6]:
clf = SVC(random_state=14)
clf.fit(X_train, y_train)
prediccion_ =clf.predict(X_test)

score_ = round(accuracy_score(y_test, prediccion_),4)
print('################# Puntuación ##################')
print('###################',score_,'####################')

#Matriz de confusión
conf_matrix = confusion_matrix(y_test, prediccion_,labels=['H1','H2','H3'])
print(pd.DataFrame(data=conf_matrix, index=['H1','H2','H3'], columns=['H1_predicción','H2_predicción','H3_predicción']))

print('')
print('')
print('#####################################################')
print('##################### Scores ########################')
print('#####################################################')
print('')

#Scores
print (classification_report(y_test, prediccion_))

################# Puntuación ##################
################### 0.7199 ####################
    H1_predicción  H2_predicción  H3_predicción
H1            474             34            114
H2            103            379            100
H3            108             46            445


#####################################################
##################### Scores ########################
#####################################################

              precision    recall  f1-score   support

          H1       0.69      0.76      0.73       622
          H2       0.83      0.65      0.73       582
          H3       0.68      0.74      0.71       599

    accuracy                           0.72      1803
   macro avg       0.73      0.72      0.72      1803
weighted avg       0.73      0.72      0.72      1803



### Ajuste de los hiperparámetros

In [7]:

grilla_parametros = {'C': [500, 1000, 2000,3000], 'gamma': [0.1 ,1, 1.1, 2], 'kernel':['rbf']} 
  
grid = GridSearchCV(SVC(random_state=14), grilla_parametros, refit = True, verbose = 3)
  
grid.fit(X_train, y_train)


print(grid.best_params_)
  
print(grid.best_estimator_)

print(grid.best_params_, ":", grid.best_score_)




Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=500, gamma=0.1, kernel=rbf ....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ........ C=500, gamma=0.1, kernel=rbf, score=0.793, total=   3.3s
[CV] C=500, gamma=0.1, kernel=rbf ....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.3s remaining:    0.0s


[CV] ........ C=500, gamma=0.1, kernel=rbf, score=0.813, total=   3.3s
[CV] C=500, gamma=0.1, kernel=rbf ....................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.6s remaining:    0.0s


[CV] ........ C=500, gamma=0.1, kernel=rbf, score=0.773, total=   3.6s
[CV] C=500, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=500, gamma=0.1, kernel=rbf, score=0.782, total=   4.1s
[CV] C=500, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=500, gamma=0.1, kernel=rbf, score=0.792, total=   3.3s
[CV] C=500, gamma=1, kernel=rbf ......................................
[CV] .......... C=500, gamma=1, kernel=rbf, score=0.872, total=   4.6s
[CV] C=500, gamma=1, kernel=rbf ......................................
[CV] .......... C=500, gamma=1, kernel=rbf, score=0.891, total=   5.8s
[CV] C=500, gamma=1, kernel=rbf ......................................
[CV] .......... C=500, gamma=1, kernel=rbf, score=0.886, total=   5.1s
[CV] C=500, gamma=1, kernel=rbf ......................................
[CV] .......... C=500, gamma=1, kernel=rbf, score=0.886, total=   5.5s
[CV] C=500, gamma=1, kernel=rbf ......................................
[CV] .

[CV] ....... C=3000, gamma=0.1, kernel=rbf, score=0.821, total=   6.3s
[CV] C=3000, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=3000, gamma=0.1, kernel=rbf, score=0.841, total=   6.7s
[CV] C=3000, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=3000, gamma=0.1, kernel=rbf, score=0.824, total=   6.5s
[CV] C=3000, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=3000, gamma=0.1, kernel=rbf, score=0.824, total=   6.1s
[CV] C=3000, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=3000, gamma=0.1, kernel=rbf, score=0.838, total=   6.1s
[CV] C=3000, gamma=1, kernel=rbf .....................................
[CV] ......... C=3000, gamma=1, kernel=rbf, score=0.882, total=  10.5s
[CV] C=3000, gamma=1, kernel=rbf .....................................
[CV] ......... C=3000, gamma=1, kernel=rbf, score=0.893, total=  10.7s
[CV] C=3000, gamma=1, kernel=rbf .....................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  8.9min finished


{'C': 2000, 'gamma': 1.1, 'kernel': 'rbf'}
SVC(C=2000, gamma=1.1, random_state=14)
{'C': 2000, 'gamma': 1.1, 'kernel': 'rbf'} : 0.8930816231787105


In [8]:
print(f"kernel: {grid.best_params_['kernel']}")
print(f"gamma: {grid.best_params_['gamma']}")
print(f"C: {grid.best_params_['C']}")

kernel: rbf
gamma: 1.1
C: 2000


In [9]:
clf_ = SVC(random_state=14, C=grid.best_params_['C'],
           gamma=grid.best_params_['gamma'],
           kernel=grid.best_params_['kernel'])

clf_.fit(X_train, y_train)

prediccion =clf_.predict(X_test)

score_ = round(accuracy_score(y_test, prediccion),4)
print('################# Puntuación ##################')
print('###################',score_,'####################')

#Matriz de confusión
conf_matrix = confusion_matrix(y_test, prediccion,labels=['H1','H2','H3'])
print(pd.DataFrame(data=conf_matrix, index=['H1','H2','H3'], columns=['H1_predicción','H2_predicción','H3_predicción']))

print('')
print('')
print('#####################################################')
print('##################### Scores ########################')
print('#####################################################')
print('')
print(classification_report(y_test, prediccion))

################# Puntuación ##################
################### 0.9079 ####################
    H1_predicción  H2_predicción  H3_predicción
H1            574             14             34
H2             30            526             26
H3             45             17            537


#####################################################
##################### Scores ########################
#####################################################

              precision    recall  f1-score   support

          H1       0.88      0.92      0.90       622
          H2       0.94      0.90      0.92       582
          H3       0.90      0.90      0.90       599

    accuracy                           0.91      1803
   macro avg       0.91      0.91      0.91      1803
weighted avg       0.91      0.91      0.91      1803

